In [ ]:
import os
from autogen import ConversableAgent, config_list_from_json, AssistantAgent, UserProxyAgent, initiate_chats
from autogen.code_utils import create_virtual_env
from autogen.coding import CodeBlock, LocalCommandLineCodeExecutor
import pprint
import json


config_list = config_list_from_json(env_or_file= "OAI_CONFIG_LIST")

data_agent_system_message = """
You have been given coding capability to solve tasks using Python code, Pandas, rapidfuzz.
In the following cases, suggest python code (in python coding blocks) for the user to execute.
    1. When you need to collect info, use the code to output the info you need, for example, browse or search the web, download/read a file, print the content of a webpage or a file, get the current date/time, check the operating system. After sufficient info is printed and the task is ready to be solved based on your language skill, you can solve the task by yourself.
    2. When you need to perform some task with code, use the code to perform the task and output the result. Finish the task smartly.
Solve the task step by step if you need to. If a plan is not provided, explain your plan first. Be clear which step uses code, and which step uses your language skill. 
When using code, you must indicate the script type in the code block. The user cannot provide any other feedback or perform any other action. The user can't modify your code. So do not suggest incomplete code which requires users to modify. Don't use a code block if it's not intended to be executed.
If you want the user to save the code in a file before executing it, put # filename: <filename> inside the code block as the first line. Don't include multiple code blocks in one response. Do not ask users to copy and paste the result. Instead, use 'print' function for the output when relevant. Check the execution result returned.
"""

checky_becky_system_message = '''Critic. You are a helpful assistant highly skilled in evaluating the quality of a given dataset by providing a score from 1 (bad-many duplicates) - 10 (no duplicates) while providing clear rationale. YOU MUST CONSIDER DATA CLEANING BEST PRACTICES for each evaluation. Specifically, you can carefully evaluate the dataset across the following dimensions
- exact duplicates: if any two elements have the same exact name, gmail, created_at, and IP. 
- names that don't have a proper encoding within an excel sheet, examples include what they should be converted to after the : (e.g, LÃ©o Lesueur bichet: Léo Lesueur bichet, MÃ¼slÃ¼m Dag: Müslüm Dağ )
- countries that don't have a proper encoding within an excel sheet, examples include what they should be converted to after the : (e.g., RÃ¼sselheim:Rüsselsheim, MÃ¼nchen: München, HÃ¼rth: Hürth, Ã–sterreich/Graz: Österreich/Graz) 
 

YOU MUST PROVIDE A SCORE for each of the above dimensions.
{exact duplicates: 0, names: 0, countries: 0}
Do not suggest code.
Take into account all misencodings that could be present within an excel sheet, and 
Finally, based on the critique above, suggest a concrete list of actions that the coder should take to improve the dataset.
Reply "TERMINATE" when somthing meets above a 7.
'''

d_agent = AssistantAgent(name="jonsnow",
llm_config={"config_list": config_list},
human_input_mode="NEVER",  # Never ask for human input.
is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("further assistance") and x.get("content", "").rstrip().endswith("Care!"),
system_message= data_agent_system_message,
max_consecutive_auto_reply=10,
)

user = UserProxyAgent(name="jbob",
llm_config={"config_list": config_list},
human_input_mode="NEVER",  # Never ask for human input.
is_termination_msg=lambda x: x.get("content", "") and x.get("content", "").rstrip().endswith("TERMINATE"),
code_execution_config={"use_docker": False, "work_dir": "work_dir3"},
)

##
##check = AssistantAgent(name = "checkybecky",
##llm_config={"config_list": config_list},
##system_message= checky_becky_system_message,
##human_input_mode="NEVER",
##code_execution_config={"use_docker": False, "work_dir": "work_dir2"},
##)



In [2]:
task1 = '''
Here are a bunch of steps:

You are also going to be given code from a coder agent. Start a virtual environment using the line: conda activate pyautogen and then proceed


1. Open the excel file found in the MemberData folder of the inputs folder of the HackMIT2024 folder. This is the "Members.xlsx" file

2. Load Relevant Tabs In This Sheet:
    -Subscriber_ID (e.g., 31872748)
    -Member_Seq (e.g., 1)
    -Last_Name (e.g., M)
    -First_Name (e.g., G)
    -Relationship (e.g., 1)
    -Birth_Date (e.g., 1972-11-23 0:00:00)
    -Date_Enrolled (e.g., 2019-03-01 0:00:00)
    -Disenroll_Date (e.g., 2019-08-31 0:00:00)
    -Sex (e.g., M, F)
    -Adult_Child (e.g., )
    -Other_Insurance (e.g., FALSE)
    -Adult_Dependent (e.g., FALSE)
    -Notes (e.g., )
    -Entry_Date (e.g., 2019-02-14 21:32:10)
    -Entry_User (e.g., MemberImport_Conifer)
    -Update_Date (e.g., 2019-09-05 21:30:06)
    -Update_User (e.g., MemberImport_Conifer)
    -Alternate_ID (e.g., LBCV1216210)
    -VIP_Flag (e.g., 0)
    -Pend_Flag (e.g., )
    -Pend_Ex_Code (e.g., )
    -Other_Name (e.g., )
    -Pre_Exist (e.g., )
    -Pre_Exist_End (e.g., )
    -Pre_Exist_Ex_Code (e.g., )
    -Student (e.g., )
    -Date_Of_Death (e.g., )
    -Marital_Status (e.g., C)
    -Ethnicity_Code (e.g., )
    -Middle_Name (e.g., M)
    -Name_Suffix (e.g., )
    -Salutation (e.g., )
    -SSN (e.g., 123456789)
    -Unique_ID (e.g., )
    -Access_Code (e.g., )
    -Student_End (e.g., )
    -Adult_Dependent_End (e.g., )
    -Height (e.g., )
    -Weight (e.g., )
    -Continue_Coverage (e.g., FALSE)
    -Continue_Coverage_Ex_Code (e.g., )
    -Continue_Coverage_End_Date (e.g., )
    -Credible_Coverage (e.g., FALSE)
    -Coverage_Type (e.g., )
    -Use_Member_Plan_Year (e.g., )
    -Plan_Year_Frequency (e.g., )
    -Plan_Year_Frequency_Type (e.g., )
    -Creditable_Coverage_Start (e.g., )
    -Creditable_Coverage_End (e.g., )
    -Initial_Volume_Salary_Pct (e.g., )
    -Initial_Volume (e.g., )
    -Smoker (e.g., FALSE)

3. Preprocessing of the excel sheet
    -First for Relationship, ENSURE that all relationships have been converted to a proper encoding format (e.g, 1: Insured, 2: Spouse, 3: Father or Mother, 4: Grandfather or Grandmother, 5: Grandson or Granddaughter, 6: Uncle or Aunt, 7: Nephew or Niece, 8: Cousin, 9: Adopted Child, 10: Foster Child, 11: Son-in-law or Daughter-in-law, 12: Brother-in-law or Sister-in-law, 13: Mother-in-law or Father-in-law, 14: Brother or Sister, 15: Ward, 16: Stepparent, 17: Stepson or Stepdaughter, 18: Sponsored Dependent, 19: Child, 20: Dependent of a Minor Dependent, 21: Ex-Spouse, 22: Guardian, 23: Court Appointed Guardian, 24: Collateral Dependent, 25: Life Partner, 26: Annuitant, 27: Trustee, 28: Other Relationship, 29: Other Relative)

4. Create A New PreProcessed DataSet:     
-Store the file in the folder called PrePreprocessed. Name the file prepped. 
-The dataset should have ONLY following Relevant Tabs:
    -Subscriber_ID (e.g., 31872748)
    -Member_Seq (e.g., 1)
    -Last_Name (e.g., M)
    -First_Name (e.g., G)
    -Relationship (e.g., Father)
    -Birth_Date (e.g., 1972-11-23 0:00:00)
    -Date_Enrolled (e.g., 2019-03-01 0:00:00)
    -Disenroll_Date (e.g., 2019-08-31 0:00:00)
    -Sex (e.g., M, F)
    -Adult_Child (e.g., )
    -Other_Insurance (e.g., FALSE)
    -Adult_Dependent (e.g., FALSE)
    -Notes (e.g., )
    -Entry_Date (e.g., 2019-02-14 21:32:10)
    -Entry_User (e.g., MemberImport_Conifer)
    -Update_Date (e.g., 2019-09-05 21:30:06)
    -Update_User (e.g., MemberImport_Conifer)
    -Alternate_ID (e.g., LBCV1216210)
    -VIP_Flag (e.g., 0)
    -Pend_Flag (e.g., )
    -Pend_Ex_Code (e.g., )
    -Other_Name (e.g., )
    -Pre_Exist (e.g., )
    -Pre_Exist_End (e.g., )
    -Pre_Exist_Ex_Code (e.g., )
    -Student (e.g., )
    -Date_Of_Death (e.g., )
    -Marital_Status (e.g., C)
    -Ethnicity_Code (e.g., )
    -Middle_Name (e.g., M)
    -Name_Suffix (e.g., )
    -Salutation (e.g., )
    -SSN (e.g., 123456789)
    -Unique_ID (e.g., )
    -Access_Code (e.g., )
    -Student_End (e.g., )
    -Adult_Dependent_End (e.g., )
    -Height (e.g., )
    -Weight (e.g., )
    -Continue_Coverage (e.g., FALSE)
    -Continue_Coverage_Ex_Code (e.g., )
    -Continue_Coverage_End_Date (e.g., )
    -Credible_Coverage (e.g., FALSE)
    -Coverage_Type (e.g., )
    -Use_Member_Plan_Year (e.g., )
    -Plan_Year_Frequency (e.g., )
    -Plan_Year_Frequency_Type (e.g., )
    -Creditable_Coverage_Start (e.g., )
    -Creditable_Coverage_End (e.g., )
    -Initial_Volume_Salary_Pct (e.g., )
    -Initial_Volume (e.g., )
    -Smoker (e.g., FALSE)


5. Open the file in the folder called PrePreprocessed. This file is called prepped.xlsx. 

6. Load Relevant Tabs In This Sheet:
    -Subscriber_ID (e.g., 31872748)
    -Member_Seq (e.g., 1)
    -Last_Name (e.g., M)
    -First_Name (e.g., G)
    -Relationship (e.g., I)
    -Birth_Date (e.g., 1972-11-23 0:00:00)
    -Date_Enrolled (e.g., 2019-03-01 0:00:00)
    -Disenroll_Date (e.g., 2019-08-31 0:00:00)
    -Sex (e.g., M, F)
    -Adult_Child (e.g., )
    -Other_Insurance (e.g., FALSE)
    -Adult_Dependent (e.g., FALSE)
    -Notes (e.g., )
    -Entry_Date (e.g., 2019-02-14 21:32:10)
    -Entry_User (e.g., MemberImport_Conifer)
    -Update_Date (e.g., 2019-09-05 21:30:06)
    -Update_User (e.g., MemberImport_Conifer)
    -Alternate_ID (e.g., LBCV1216210)
    -VIP_Flag (e.g., 0)
    -Pend_Flag (e.g., )
    -Pend_Ex_Code (e.g., )
    -Other_Name (e.g., )
    -Pre_Exist (e.g., )
    -Pre_Exist_End (e.g., )
    -Pre_Exist_Ex_Code (e.g., )
    -Student (e.g., )
    -Date_Of_Death (e.g., )
    -Marital_Status (e.g., C)
    -Ethnicity_Code (e.g., )
    -Middle_Name (e.g., M)
    -Name_Suffix (e.g., )
    -Salutation (e.g., )
    -SSN (e.g., 123456789)
    -Unique_ID (e.g., )
    -Access_Code (e.g., )
    -Student_End (e.g., )
    -Adult_Dependent_End (e.g., )
    -Height (e.g., )
    -Weight (e.g., )
    -Continue_Coverage (e.g., FALSE)
    -Continue_Coverage_Ex_Code (e.g., )
    -Continue_Coverage_End_Date (e.g., )
    -Credible_Coverage (e.g., FALSE)
    -Coverage_Type (e.g., )
    -Use_Member_Plan_Year (e.g., )
    -Plan_Year_Frequency (e.g., )
    -Plan_Year_Frequency_Type (e.g., )
    -Creditable_Coverage_Start (e.g., )
    -Creditable_Coverage_End (e.g., )
    -Initial_Volume_Salary_Pct (e.g., )
    -Initial_Volume (e.g., )
    -Smoker (e.g., FALSE)


7. Preprocessing of the excel sheet
    -First for Relationship, ENSURE that all relationships have been converted to a proper encoding format (e.g, Insured: I, Spouse: S, Child: D, Father or Mother: D, Grandfather or Grandmother: D, Grandson or Granddaughter: D, Uncle or Aunt: D, Nephew or Niece: D, Cousin: D, Adopted Child: D, Foster Child: D, Son-in-law or Daughter-in-law: D, Brother-in-law or Sister-in-law: D, Mother-in-law or Father-in-law: D, Brother or Sister: D, Ward: D, Stepparent: D, Stepson or Stepdaughter: D, Sponsored Dependent: D, Dependent of a Minor Dependent: D, Ex-Spouse: D, Guardian: D, Court Appointed Guardian: D, Collateral Dependent: D, Life Partner: D, Annuitant: D, Trustee: D, Other Relationship: D, Other Relative: D)
    -Second for Birth_Date, ENSURE that the dates follow the YYYYMMDD format (e.g, 11/23/1972 0:00:00 -> 19721123, 2/27/1988 0:00:00 -> 19880227, 1/7/1968 0:00:00 -> 19680107)
    -Third for Disenroll_Date, ENSURE that the dates follow the YYYYMMDD format (e.g, 11/23/1972 0:00:00 -> 19721123, 2/27/1988 0:00:00 -> 19880227, 1/7/1968 0:00:00 -> 19680107)
    -Fourth for Date_Enrolled, ENSURE that the dates follow the YYYYMMDD format (e.g, 11/23/1972 0:00:00 -> 19721123, 2/27/1988 0:00:00 -> 19880227, 1/7/1968 0:00:00 -> 19680107)
)

8. Create A New PreProcessed DataSet:     
-Store the file in the folder called PrePreprocessed. Name the file finished. 

9. Document the Process:
        - Print the first few rows of each dataset (Record Type, Group Number, Division, Insured I.D., Member Number, Person Code, Relationship, Last Name, First Name, Middle_Initial, Date of Birth, Gender, Address 1, Address 2, Member Number Overflow, Miscellaneous Code 8, Insured I.D. Overflow, Filler 1, Alternate Member Number, PCP Start Date, PCP End Date, Medical Group Start Date, Medical Group End Date, Medical Group, Reserved 1....) to verify the content.
        - Document any potential exceptions and ensure the process runs smoothly.
        - Provide a summary of the results, including any rows met the duplicate threshold and any cases requiring manual review or further action.

        Example Output:
        Make sure to handle any potential exceptions and ensure the process runs smoothly. Document the steps and provide a summary of the results.
'''

In [3]:
task2 = '''Here are a bunch of steps:

You are also going to be given code from a coder agent. Start a virtual environment using the line: conda activate pyautogen and then proceed

1. Open the excel file found in the HackMIT2024 results folder. This is the "Results_Not_Formated.xlsx" file

2. Load Relevant Tabs In This Sheet:
    - Record Type (e.g., 23)  
    - Group Number (e.g., CSFDC1)  
    - Division (e.g., CSFDC)  
    - Insured I.D. (e.g., AGCS12379)  
    - Member Number (e.g., AGCS1237910)  
    - Person Code (e.g., 00)  
    - Relationship (e.g., I)  
    - Last Name (e.g., K)  
    - First Name (e.g., J)  
    - Middle Initial (e.g., W)  
    - Date of Birth (e.g., 19750902)  
    - Gender (e.g., M)  
    - Address 1 (e.g., 1234 Sunset Blvd)  
    - Address 2 (e.g., Apt 2)  
    - Member Number Overflow (e.g., )  
    - Miscellaneous Code 8 (e.g., )  
    - Insured I.D. Overflow (e.g., )  
    - Filler 1 (e.g., )  
    - Alternate Member Number (e.g., )  
    - PCP Start Date (e.g., 20240101)  
    - PCP End Date (e.g., )  
    - Medical Group Start Date (e.g., )  
    - Medical Group End Date (e.g., )  
    - Medical Group (e.g., KPC07)  
    - Reserved 1 (e.g., )  
    - Reserved 2 (e.g., )  
    - Miscellaneous Code 1 (e.g., )  
    - Miscellaneous Code 2 (e.g., )  
    - Miscellaneous Code 3 (e.g., )  
    - Miscellaneous Code 4 (e.g., )  
    - City (e.g., MILLSBORO)  
    - State (e.g., DE)  
    - Zip Code (e.g., 19966)  
    - SSN (e.g., 123456789)  
    - Dependent Coverage Code (e.g., 1)  
    - COB Code (e.g., )  
    - Effective Date (e.g., 20240101)  
    - Term Date (e.g., )  
    - Headquarters Code (e.g., )  
    - PCP ID Number (e.g., )  
    - Miscellaneous Code 5 (e.g., )  
    - Miscellaneous Code 6 (e.g., )  
    - Miscellaneous Code 7 (e.g., )  
    - Filler 2 (e.g., )  
    - HIC Number (e.g., )  
    - ID Card Print Date (e.g., )  
    - Medicare Code (e.g., )  
    - Term Code (e.g., )  
    - Force ID Card (e.g., )  
    - Credit Card Number (e.g., )  
    - Expiration Date (e.g., )  
    - Miscellaneous Code 9 (e.g., )  
    - Miscellaneous Code 10 (e.g., )  
    - Miscellaneous Code 11 (e.g., )  
    - Miscellaneous Code 12 (e.g., )  
    - Reserved Xref Member Number (e.g., AGCS12379)  
    - Reserved Xref Family Position (e.g., 10)  
    - COB Prime Code (e.g., )  
    - Medicaid Number (e.g., )  
    - Filler 3 (e.g., )  
    - PCP ID Overflow (e.g., )  
    - Void Flag (e.g., )  
    - Confidential Communications Flag (e.g., )  
    - Primary Phone Number (e.g., )  
    - Primary Phone Number Extension (e.g., )  
    - Secondary Phone Number (e.g., )  
    - Secondary Phone Number Extension (e.g., )  
    - Medicare ID (e.g., )  
    - Reserved Filler 1 (e.g., )  
    - Foreign Language Preference (e.g., )  
    - Ethnicity (e.g., )  
    - Filler 4 (e.g., )  
    - Reserved Filler 2 (e.g., )

    3. In the folder called PrePreprocessed. Open the file named finished.xlsx.

    4. Load Relevant Tabs In This Sheet:
        -Subscriber_ID (e.g., 31872748)
        -Sex (e.g., M, F)
        -SSN (e.g, 123456789)
        -Birth_Date (e.g, 19750824, 20230729, 19950222, 19611103)
        -Date_Enrolled (e.g., 19750824)
        -Disenroll_Date (e.g., 19750824)
        -Alternate_ID (e.g., LBCV1216210)
        -Last_Name (e.g, F, M, B, S)
        -First_Name(e.g, G, S, K, R, M, C)

    5. Match of Member Number and Alternate_ID:
    
    Go through all Member Numbers and see if there is a match with a value in Alternate_ID. If there is a match then in that row :
    -find the corresponding Subscriber_ID (e.g., 31872748) and populate the column
    -if empty, populate the Gender column of that row with the Sex of that row (e.g., M, F)
    -if empty, populate with the corresponding SSN (e.g, 123456789)
    -if empty, populate the Effective Date with Date_Enrolled (e.g., 20240101)
    -if empty, populate the Term Date column with Disenroll_Date (e.g., 19750824)
    -if empty, populate the Last Name Column of the row with Last_Name (e.g, F, M, B, S)
    -if empty, populate the Date of Birth Column of the row with Birth_Date (e.g, 19750824, 20230729, 19950222, 19611103)

    and save it into the new dataset in step 7.
    
    6. Add on New Alternate_ID from finished.xlsx:
    Add a new row and in the row to the next dataset:
    -populate the Member Number column with the Alternate_ID data (e.g, LBCV1216210, AGCS1238012, AGCS1248912, AGKR1206813, AGMN1214210)
    -populate with corresponding the Subscriber_ID data (e.g., 31872748) 
    -populate the Gender column with the corresponding column Sex data (e.g, M, F) 
    -SSN data (e.g, 123456789)
    -populate the Effective Date with Date_Enrolled (e.g., 20240101)
    -populate the Term Date column with Disenroll_Date (e.g., 19750824)
    -populate the Date of Birth Column of the row with correspoding Birth_Date of that Subscriber_ID data (e.g, 19750824)
    -populate the Last Name Column of the row with Last_Name (e.g, F, M, B, S)
    -populate the Date of Birth Column of the row with Birth_Date (e.g, 19750824, 20230729, 19950222, 19611103)

    and save it into the new dataset in step 7. 

    7.  Create a New Dataset:
    The dataset will have the following tabs and be saved in FirstOutput. Name the dataset first_fill. 
        - Record Type (e.g., 23)  
        - Group Number (e.g., CSFDC1)  
        - Division (e.g., CSFDC)  
        - Insured I.D. (e.g., AGCS12379)  
        - Member Number (e.g., AGCS1237910)  
        - Subscriber_ID (e.g., 31872748)
        - Person Code (e.g., 00)  
        - Relationship (e.g., I)  
        - Last Name (e.g., K)  
        - First Name (e.g., J)  
        - Middle Initial (e.g., W)  
        - Date of Birth (e.g., 19750902)  
        - Gender (e.g., M, F)  
        - Address 1 (e.g., 1234 Sunset Blvd)  
        - Address 2 (e.g., Apt 2)  
        - Member Number Overflow (e.g., )  
        - Miscellaneous Code 8 (e.g., )  
        - Insured I.D. Overflow (e.g., )  
        - Filler 1 (e.g., )  
        - Alternate Member Number (e.g., )  
        - PCP Start Date (e.g., 20240101)  
        - PCP End Date (e.g., )  
        - Medical Group Start Date (e.g., )  
        - Medical Group End Date (e.g., )  
        - Medical Group (e.g., KPC07)  
        - Reserved 1 (e.g., )  
        - Reserved 2 (e.g., )  
        - Miscellaneous Code 1 (e.g., )  
        - Miscellaneous Code 2 (e.g., )  
        - Miscellaneous Code 3 (e.g., )  
        - Miscellaneous Code 4 (e.g., )  
        - City (e.g., MILLSBORO)  
        - State (e.g., DE)  
        - Zip Code (e.g., 19966)  
        - SSN (e.g., 123456789)  
        - Dependent Coverage Code (e.g., 1)  
        - COB Code (e.g., )  
        - Effective Date (e.g., 20240101)  
        - Term Date (e.g., )  
        - Headquarters Code (e.g., )  
        - PCP ID Number (e.g., )  
        - Miscellaneous Code 5 (e.g., )  
        - Miscellaneous Code 6 (e.g., )  
        - Miscellaneous Code 7 (e.g., )  
        - Filler 2 (e.g., )  
        - HIC Number (e.g., )  
        - ID Card Print Date (e.g., )  
        - Medicare Code (e.g., )  
        - Term Code (e.g., )  
        - Force ID Card (e.g., )  
        - Credit Card Number (e.g., )  
        - Expiration Date (e.g., )  
        - Miscellaneous Code 9 (e.g., )  
        - Miscellaneous Code 10 (e.g., )  
        - Miscellaneous Code 11 (e.g., )  
        - Miscellaneous Code 12 (e.g., )  
        - Reserved Xref Member Number (e.g., AGCS12379)  
        - Reserved Xref Family Position (e.g., 10)  
        - COB Prime Code (e.g., )  
        - Medicaid Number (e.g., )  
        - Filler 3 (e.g., )  
        - PCP ID Overflow (e.g., )  
        - Void Flag (e.g., )  
        - Confidential Communications Flag (e.g., )  
        - Primary Phone Number (e.g., )  
        - Primary Phone Number Extension (e.g., )  
        - Secondary Phone Number (e.g., )  
        - Secondary Phone Number Extension (e.g., )  
        - Medicare ID (e.g., )  
        - Reserved Filler 1 (e.g., )  
        - Foreign Language Preference (e.g., )  
        - Ethnicity (e.g., )  
        - Filler 4 (e.g., )  
        - Reserved Filler 2 (e.g., )

    7. Document the Process:
        - Print the first few rows of each dataset (Record Type, Group Number, Division, Insured I.D., Member Number, Person Code, Relationship, Last Name, First Name, Middle_Initial, Date of Birth, Gender, Address 1, Address 2, Member Number Overflow, Miscellaneous Code 8, Insured I.D. Overflow, Filler 1, Alternate Member Number, PCP Start Date, PCP End Date, Medical Group Start Date, Medical Group End Date, Medical Group, Reserved 1....) to verify the content.
        - Document any potential exceptions and ensure the process runs smoothly.
        - Provide a summary of the results, including any rows met the duplicate threshold and any cases requiring manual review or further action.

        Example Output:
        Make sure to handle any potential exceptions and ensure the process runs smoothly. Document the steps and provide a summary of the results.
'''





In [4]:
task3 = '''Here are a bunch of steps:

You are also going to be given code from a coder agent. Start a virtual environment using the line: conda activate pyautogen and then proceed

1.Open the excel file found in the FirstOutput results folder. This is the "first_fill.xlsx" file

2. Load the Relevant Tabs In This Sheet:
        - Record Type (e.g., 23)  
        - Group Number (e.g., CSFDC1)  
        - Division (e.g., CSFDC)  
        - Insured I.D. (e.g., AGCS12379)  
        - Member Number (e.g., AGCS1237910)  
        - Subscriber_ID (e.g., 31872748)
        - Person Code (e.g., 00)  
        - Relationship (e.g., I)  
        - Last Name (e.g., K)  
        - First Name (e.g., J)  
        - Middle Initial (e.g., W)  
        - Date of Birth (e.g., 19750902)  
        - Gender (e.g., M)  
        - Address 1 (e.g., 1234 Sunset Blvd)  
        - Address 2 (e.g., Apt 2)  
        - Member Number Overflow (e.g., )  
        - Miscellaneous Code 8 (e.g., )  
        - Insured I.D. Overflow (e.g., )  
        - Filler 1 (e.g., )  
        - Alternate Member Number (e.g., )  
        - PCP Start Date (e.g., 20240101)  
        - PCP End Date (e.g., )  
        - Medical Group Start Date (e.g., )  
        - Medical Group End Date (e.g., )  
        - Medical Group (e.g., KPC07)  
        - Reserved 1 (e.g., )  
        - Reserved 2 (e.g., )  
        - Miscellaneous Code 1 (e.g., )  
        - Miscellaneous Code 2 (e.g., )  
        - Miscellaneous Code 3 (e.g., )  
        - Miscellaneous Code 4 (e.g., )  
        - City (e.g., MILLSBORO)  
        - State (e.g., DE)  
        - Zip Code (e.g., 19966)  
        - SSN (e.g., 123456789)  
        - Dependent Coverage Code (e.g., 1)  
        - COB Code (e.g., )  
        - Effective Date (e.g., 20240101)  
        - Term Date (e.g., )  
        - Headquarters Code (e.g., )  
        - PCP ID Number (e.g., )  
        - Miscellaneous Code 5 (e.g., )  
        - Miscellaneous Code 6 (e.g., )  
        - Miscellaneous Code 7 (e.g., )  
        - Filler 2 (e.g., )  
        - HIC Number (e.g., )  
        - ID Card Print Date (e.g., )  
        - Medicare Code (e.g., )  
        - Term Code (e.g., )  
        - Force ID Card (e.g., )  
        - Credit Card Number (e.g., )  
        - Expiration Date (e.g., )  
        - Miscellaneous Code 9 (e.g., )  
        - Miscellaneous Code 10 (e.g., )  
        - Miscellaneous Code 11 (e.g., )  
        - Miscellaneous Code 12 (e.g., )  
        - Reserved Xref Member Number (e.g., AGCS12379)  
        - Reserved Xref Family Position (e.g., 10)  
        - COB Prime Code (e.g., )  
        - Medicaid Number (e.g., )  
        - Filler 3 (e.g., )  
        - PCP ID Overflow (e.g., )  
        - Void Flag (e.g., )  
        - Confidential Communications Flag (e.g., )  
        - Primary Phone Number (e.g., )  
        - Primary Phone Number Extension (e.g., )  
        - Secondary Phone Number (e.g., )  
        - Secondary Phone Number Extension (e.g., )  
        - Medicare ID (e.g., )  
        - Reserved Filler 1 (e.g., )  
        - Foreign Language Preference (e.g., )  
        - Ethnicity (e.g., )  
        - Filler 4 (e.g., )  
        - Reserved Filler 2 (e.g., )

    3. Open the excel file called found in the MemberData found in inputs folder in the HackMIT2024 folder. This is  "Subscribers.csv" file

    4. Load the Relevant Tabs In This Sheet:
        -Subscriber_ID (e.g, 197545187, 544567590, 733549125, 797621378, 823509195, 723494588, 888420157, 654476212, 97210220, 861305712, 52805809)
        -Address (e.g, 1234 Sunset Blvd) 
        -Address2 (e.g, Apt 2)
        -Zip_Code (e.g, 25430, 17540, 21502, 19973, 18951, 19446, 19144)

    5. Go through all Subscriber_ID and see if there is a match with Subscriber_ID in "Subscribes.csv". If there is a match then in that row:
    -if empty, populate the Address 1 column of that row with the Address of that row (e.g. 1234 Sunset Blvd)
    -if empty, populate Address 2 column with the corresponding Address2 (e.g, Apt 2)
    -if empty, populate the Zip Code column with Zip Code (e.g., 25430)
    -if empty, populate the Last Name Column of the row with Last_Name (e.g, F, M, B, S)
    -if empty, populate the Date of Birth Column of the row with Birth_Date (e.g, 19750824, 20230729, 19950222, 19611103)
    
    If there is no match in the row, add a new row and in the row:
    -populate the Member Number with the Alternate_ID data (e.g, AGCS1237910, AGCS1238012, AGCS1248912)
    -populate with corresponding the Subscriber_ID data (e.g., 31872748) 
    -populate the Gender column with the corresponding column Sex data (e.g, M, F) 
    -SSN data (e.g, 123456789)
    -populate the Term Date column with Disenroll_Date (e.g., 19750824)
    -populate the Date of Birth Column of the row with correspoding Birth_Date of that Subscriber_ID data (e.g, 19750824)
    -populate the Last Name Column of the row with Last_Name (e.g, F, M, B, S)
    -populate the Date of Birth Column of the row with Birth_Date (e.g, 19750824, 20230729, 19950222, 19611103)

    and save it into the new dataset in step 6. 

    6. Create a new Dataset:
    The dataset will have the following tabs and be saved in SecondOutput. Call the dataset second_fill. 
        - Record Type (e.g., 23)  
        - Group Number (e.g., CSFDC1)  
        - Division (e.g., CSFDC)  
        - Insured I.D. (e.g., AGCS12379)  
        - Member Number (e.g., AGCS1237910)  
        - Subscriber_ID (e.g., 31872748)
        - Person Code (e.g., 00)  
        - Relationship (e.g., I)  
        - Last Name (e.g., K)  
        - First Name (e.g., J)  
        - Middle Initial (e.g., W)  
        - Date of Birth (e.g., 19750902)  
        - Gender (e.g., M)  
        - Address 1 (e.g., 1234 Sunset Blvd)  
        - Address 2 (e.g., Apt 2)  
        - Member Number Overflow (e.g., )  
        - Miscellaneous Code 8 (e.g., )  
        - Insured I.D. Overflow (e.g., )  
        - Filler 1 (e.g., )  
        - Alternate Member Number (e.g., )  
        - PCP Start Date (e.g., 20240101)  
        - PCP End Date (e.g., )  
        - Medical Group Start Date (e.g., )  
        - Medical Group End Date (e.g., )  
        - Medical Group (e.g., KPC07)  
        - Reserved 1 (e.g., )  
        - Reserved 2 (e.g., )  
        - Miscellaneous Code 1 (e.g., )  
        - Miscellaneous Code 2 (e.g., )  
        - Miscellaneous Code 3 (e.g., )  
        - Miscellaneous Code 4 (e.g., )  
        - City (e.g., MILLSBORO)  
        - State (e.g., DE)  
        - Zip Code (e.g., 19966)  
        - SSN (e.g., 123456789)  
        - Dependent Coverage Code (e.g., 1)  
        - COB Code (e.g., )  
        - Effective Date (e.g., 20240101)  
        - Term Date (e.g., )  
        - Headquarters Code (e.g., )  
        - PCP ID Number (e.g., )  
        - Miscellaneous Code 5 (e.g., )  
        - Miscellaneous Code 6 (e.g., )  
        - Miscellaneous Code 7 (e.g., )  
        - Filler 2 (e.g., )  
        - HIC Number (e.g., )  
        - ID Card Print Date (e.g., )  
        - Medicare Code (e.g., )  
        - Term Code (e.g., )  
        - Force ID Card (e.g., )  
        - Credit Card Number (e.g., )  
        - Expiration Date (e.g., )  
        - Miscellaneous Code 9 (e.g., )  
        - Miscellaneous Code 10 (e.g., )  
        - Miscellaneous Code 11 (e.g., )  
        - Miscellaneous Code 12 (e.g., )  
        - Reserved Xref Member Number (e.g., AGCS12379)  
        - Reserved Xref Family Position (e.g., 10)  
        - COB Prime Code (e.g., )  
        - Medicaid Number (e.g., )  
        - Filler 3 (e.g., )  
        - PCP ID Overflow (e.g., )  
        - Void Flag (e.g., )  
        - Confidential Communications Flag (e.g., )  
        - Primary Phone Number (e.g., )  
        - Primary Phone Number Extension (e.g., )  
        - Secondary Phone Number (e.g., )  
        - Secondary Phone Number Extension (e.g., )  
        - Medicare ID (e.g., )  
        - Reserved Filler 1 (e.g., )  
        - Foreign Language Preference (e.g., )  
        - Ethnicity (e.g., )  
        - Filler 4 (e.g., )  
        - Reserved Filler 2 (e.g., )

    7. Document the Process:
        - Print the first few rows of each dataset (Record Type, Group Number, Division, Insured I.D., Member Number, Person Code, Relationship, Last Name, First Name, Middle_Initial, Date of Birth, Gender, Address 1, Address 2, Member Number Overflow, Miscellaneous Code 8, Insured I.D. Overflow, Filler 1, Alternate Member Number, PCP Start Date, PCP End Date, Medical Group Start Date, Medical Group End Date, Medical Group, Reserved 1....) to verify the content.
        - Document any potential exceptions and ensure the process runs smoothly.
        - Provide a summary of the results, including any rows met the duplicate threshold and any cases requiring manual review or further action.

        Example Output:
        Make sure to handle any potential exceptions and ensure the process runs smoothly. Document the steps and provide a summary of the results.


'''

In [ ]:
response = initiate_chats(
    [
        {
            "sender": user,
            "recipient": d_agent,
            "message": task1,
            "clear_history": False,
            "silent": False,
            "summary_method": None,
        },
        {
            "sender": user,
            "recipient": d_agent,
            "message": task2,
            "clear_history": False,
            "silent": False,
            "summary_method": None,
        },
        {
            "sender": user,
            "recipient": d_agent,
            "message": task3,
            "clear_history": False,
            "silent": False,
            "summary_method": None,
        }

    ])    